# ALIBI

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from xgboost.sklearn import XGBClassifier

model = XGBClassifier()

In [13]:
model.load_model("../../models/xgboost.json")

In [4]:
from alibi.explainers import Counterfactual

In [52]:
shape = (1, 23)

In [82]:
predict_fn = lambda x: model.predict_proba(x)

In [83]:
import tensorflow as tf

tf.keras.backend.clear_session()
tf.compat.v1.disable_eager_execution()

In [84]:
cf = Counterfactual(predict_fn, shape, distance_fn='l1', target_proba=0.75,
                    target_class="other", max_iter=10000, early_stop=50, lam_init=1e-1,
                    max_lam_steps=50, tol=0.1, learning_rate_init=0.1,
                    feature_range=(-20, 1000), eps=0.01, init='identity',
                    decay=False, write_dir=None, debug=False)

In [85]:
import numpy as np

X = np.array(
    [65, 43, 2, 26, 15, 1, 1, 100, -7, 7, 8, 17, 2, 25, 0, 4, 4, 31, 83, 5, 1, 0, 75]
).reshape(1, -1)

In [86]:
explanation = cf.explain(X)

No appropriate lambda range found, try decreasing lam_init
